![Trump vs Biden](https://therealdeal.com/wp-content/uploads/2020/09/1200-Will-real-estate-take-the-stage-in-the-first-presidential-debate_-705x439.jpg)
# **Introduction**
Hi! 
Here I'll try to find interesting sides of the past USA election in tweets dedicated to two major candidates: Donald Trump and Joe Biden. 
I hope, it'll be an interesting journey. If you'll find it useful, please, **upvote** and **leave a comment**. 
Work is still in progress, so I'll be improving and expanding my analysis. Maybe I'll find some new interesting tasks. 

Now, let's start!

### **Notebook changes:**
**Version 5:** fixed errors in names of some figures; added comments to some parts of the code.

In [ ]:
# basic packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# geodata
import geopandas as gpd
from shapely.geometry import Point

# custom y-axis
from matplotlib.ticker import FuncFormatter
def millions(x, pos):
    return '%1.1fM' % (x * 1e-6)

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

# NLP
import unicodedata
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# kaggle workspace
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Loading the data
trump = pd.read_csv('../input/us-election-2020-tweets/hashtag_donaldtrump.csv', 
                    lineterminator='\n', parse_dates=True)
biden = pd.read_csv('../input/us-election-2020-tweets/hashtag_joebiden.csv', 
                    lineterminator='\n', parse_dates=True)
print('Trump dataset shape: {}'.format(trump.shape))
print('Biden dataset shape: {}'.format(biden.shape))

### A first look at the data

In [ ]:
# Trump dataset general information
trump.info()

In [ ]:
# Biden dataset general information
biden.info()

In [ ]:
# The sample of data
trump.head(3)

In [ ]:
# The sample of data
biden.head(3)

# **EDA**
"US Election 2020 Tweets" dataset is an NLP task, so the EDA algorithm includes visualizations of interesting and important numeric features (like the amount of NaN values or number of tweets per day), and detailed analysis of tweets text (length, stopwords, sentiment analysis, etc.). Tweets have a very messy structure (with emojis, hashtags, and so on), so we have to clean it up first.

In [ ]:
trump_nan = pd.Series(trump.isna().sum()[trump.isna().sum() > 0].
                      sort_values(ascending = False))
biden_nan = pd.Series(biden.isna().sum()[biden.isna().sum() > 0].
                      sort_values(ascending = False))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 5))
sns.set_style("whitegrid")
fig.suptitle('NaN values', size = 15)

sns.barplot(y = trump_nan.index, x = [len(trump)] * len(trump_nan),
            edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax1)
sns.barplot(y = trump_nan.index, x = trump_nan, 
            edgecolor = 'black', alpha = 0.8, ax = ax1,
            palette = sns.color_palette("viridis", len(trump_nan)))
ax1.get_xaxis().get_major_formatter().set_scientific(False)
ax1.set_title('Trump dataset', size = 13)


sns.barplot(y = biden_nan.index, x = [len(biden)] * len(biden_nan),
            edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax2)
sns.barplot(y = biden_nan.index, x = biden_nan, 
            edgecolor = 'black', alpha = 0.8, ax = ax2,
            palette = sns.color_palette("viridis", len(biden_nan)))
ax2.get_xaxis().get_major_formatter().set_scientific(False)
ax2.set_title('Biden dataset', size = 13)

sns.despine()

We see that geographical information (city, state, country, etc.) is mostly unknown, so this data can't be used for analysis. We'll look only at the country feature for a small insight into the distribution of users by countries, and will visualize the geodata (lat and long).

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(14, 5))

sns.kdeplot(trump['likes'], label = 'Trump', shade = True, color = 'red')
sns.kdeplot(biden['likes'], label = 'Biden', shade = True, color = 'blue')
plt.title('Distributions of likes', size = 15)
plt.legend(prop={'size': 14})
plt.show()

There are more tweets dedicated to Donald Trump than Joe Biden, but most of them have less than 6k likes. The most popular tweet has around 74k likes. The most popular tweets dedicated to Joe Biden have 100k+ likes. Let's look at them.

In [ ]:
# the top of #donaldtrump tweets by likes
trump.sort_values('likes', ascending = False)[:5]

In [ ]:
# the top of #joebiden tweets by likes
biden.sort_values('likes', ascending = False)[:5]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
sns.set_style("whitegrid")
fig.suptitle("The dependence of retweets on likes", size = 15)

sns.scatterplot(x = trump['likes'], y = trump['retweet_count'],
                color = 'red', ax = ax1)
ax1.get_yaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('Likes')
ax1.set_ylabel('Retweets')
ax1.set_title('Trump', size = 13)


sns.scatterplot(x = biden['likes'], y = biden['retweet_count'],
                color = 'blue', ax = ax2)
ax2.get_yaxis().get_major_formatter().set_scientific(False)
ax2.set_xlabel('Likes')
ax2.set_ylabel('Retweets')
ax2.set_title('Biden', size = 13)

fig.show()

In [ ]:
print('Correlation between likes and retweets (Trump): {}'.
      format(trump['likes'].corr(trump['retweet_count'])))
print('Correlation between likes and retweets (Biden): {}'.
      format(biden['likes'].corr(biden['retweet_count'])))

There is a strong positive correlation between likes and retweets that looks logical.

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(17, 5))

ax = sns.lineplot(data = pd.to_datetime(trump.created_at).dt.date.value_counts(), 
                  label = 'Trump', color = 'red', linewidth = 3)
ax = sns.lineplot(data = pd.to_datetime(biden.created_at).dt.date.value_counts(), 
                  label = 'Biden', color = 'blue', linewidth = 3)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %b'))

plt.title('Tweets amount changing', size = 15)
plt.legend(prop={'size': 14})
plt.ylabel('Tweets')
plt.show()

The chart above shows that throughout the study period Donald Trump was a more popular target for tweets, but after November 6 this situation changed. It's related to election results and Joe Biden's victory. Two peaks in activity look logical: October 23 - the day after debate day, and November 4 - the day after Election Day.

In [ ]:
trump_tweets_countries = trump.country.value_counts()[:10]
biden_tweets_countries = biden.country.value_counts()[:10]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 5))
sns.set_style("whitegrid")
fig.suptitle("Tweet authors by countries", size = 15)

sns.barplot(y = trump_tweets_countries.index, 
            x = [len(trump)] * len(trump_tweets_countries),
            edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax1)
sns.barplot(y = trump_tweets_countries.index, 
            x = trump_tweets_countries, 
            edgecolor = 'black', color = 'red', alpha = 0.8, ax = ax1)
ax1.get_xaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('')
ax1.set_title('Trump', size = 13)


sns.barplot(y = biden_tweets_countries.index, 
            x = [len(biden)] * len(biden_tweets_countries),
           edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax2)
sns.barplot(y = biden_tweets_countries.index, 
            x = biden_tweets_countries, 
            edgecolor = 'black', color = 'blue', alpha = 0.8, ax = ax2)
ax2.get_xaxis().get_major_formatter().set_scientific(False)
ax2.set_xlabel('')
ax2.set_title('Biden', size = 13)

sns.despine()

There are inconsistent country names in the data: "United States of America" and "United States", which are the same. Let's clean it up.

In [ ]:
trump['country'] = trump['country'].replace(['United States of America', 
                                             'United States'], 'USA')
biden['country'] = biden['country'].replace(['United States of America', 
                                             'United States'], 'USA')

In [ ]:
trump_tweets_countries = trump.country.value_counts()[:10]
biden_tweets_countries = biden.country.value_counts()[:10]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 5))
sns.set_style("whitegrid")
fig.suptitle("Tweet authors by countries", size = 15)

sns.barplot(y = trump_tweets_countries.index, x = [len(trump)] * len(trump_tweets_countries),
            edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax1)
sns.barplot(y = trump_tweets_countries.index, x = trump_tweets_countries, 
            edgecolor = 'black', color = 'red', alpha = 0.8, ax = ax1)
ax1.get_xaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('')
ax1.set_title('Trump', size = 13)


sns.barplot(y = biden_tweets_countries.index, x = [len(biden)] * len(biden_tweets_countries),
           edgecolor = 'black', color = 'white', alpha = 0.6, ax = ax2)
sns.barplot(y = biden_tweets_countries.index, x = biden_tweets_countries, 
            edgecolor = 'black', color = 'blue', alpha = 0.8, ax = ax2)
ax2.get_xaxis().get_major_formatter().set_scientific(False)
ax2.set_xlabel('')
ax2.set_title('Biden', size = 13)

sns.despine()

In [ ]:
# preparation of the geodata
tmp_tr = trump[['lat', 'long']].dropna()
tmp_bi = biden[['lat', 'long']].dropna()

geometry_tr = [Point(xy) for xy in zip(tmp_tr['long'], tmp_tr['lat'])]
geometry_bi = [Point(xy) for xy in zip(tmp_bi['long'], tmp_bi['lat'])]

geo_df_tr = gpd.GeoDataFrame(geometry = geometry_tr)
geo_df_bi = gpd.GeoDataFrame(geometry = geometry_bi)

wmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), facecolor = 'white')
plt.text(x = -325, y = 120, s = "The geodata of tweets", fontsize = 15)

wmap.plot(ax = ax1, edgecolors='black', color = 'white')
geo_df_tr.plot(ax = ax1, markersize = 0.5, color = 'red')
ax1.set_title('Trump', size = 13)
ax1.axis('off')

wmap.plot(ax = ax2, edgecolors='black', color = 'white')
geo_df_bi.plot(ax = ax2, markersize = 0.5, color = 'blue')
ax2.set_title('Biden', size = 13)
ax2.axis('off')

fig.show()

Note that these features aren't useful for deep analysis because over 50% of the data is NaN and give only a small insight into the distribution of tweets by countries. More useful and interesting can be the 'user_name' feature as a list of candidates' upholders (or haters).

In [ ]:
# Trump tweets authors
trump_count = pd.DataFrame(trump['user_name'].value_counts())
trump_count = pd.DataFrame({'user_name': trump_count.index, 
                            'count': trump_count.user_name})
trump_likes = trump[['user_name', 'likes']].groupby('user_name').sum()
trump_agg = pd.merge(trump_count, trump_likes, on = 'user_name', 
                     how = 'left')

# Biden tweets authors
biden_count = pd.DataFrame(biden['user_name'].value_counts())
biden_count = pd.DataFrame({'user_name': biden_count.index, 
                            'count': biden_count.user_name})
biden_likes = biden[['user_name', 'likes']].groupby('user_name').sum()
biden_agg = pd.merge(biden_count, biden_likes, on = 'user_name', 
                     how = 'left')

In [ ]:
# The dependence of likes sums on tweets amounts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
sns.set_style("whitegrid")
fig.suptitle("The dependence of likes sums on tweets amounts", size = 15)

sns.scatterplot(x = trump_agg['count'], y = trump_agg['likes'],
                color = 'red', ax = ax1)
ax1.get_yaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('Amount of tweets')
ax1.set_ylabel('Sum of likes')
ax1.set_title('Trump', size = 13)


sns.scatterplot(x = biden_agg['count'], y = biden_agg['likes'],
                color = 'blue', ax = ax2)
ax2.get_yaxis().get_major_formatter().set_scientific(False)
ax2.set_xlabel('Amount of tweets')
ax2.set_ylabel('Sum of likes')
ax2.set_title('Biden', size = 13)

fig.show()

In [ ]:
print('Average number of likes per tweet (Trump): {}'.
      format(round(trump_agg['likes'].sum() / trump_agg['count'].sum(), 0)))
print('Average number of likes per tweet (Biden): {}'.
      format(round(biden_agg['likes'].sum() / biden_agg['count'].sum(), 0)))

Tweets dedicated to Joe Biden liked more often.

In [ ]:
# User followers count
trump_user_followers = trump[['user_name', 
                              'user_followers_count']].groupby('user_name').max()
biden_user_followers = biden[['user_name', 
                              'user_followers_count']].groupby('user_name').max()

trump_top_authors_by_count = pd.merge(trump_agg[:10], trump_user_followers, 
                                      on = 'user_name', how = 'left')
biden_top_authors_by_count = pd.merge(biden_agg[:10], biden_user_followers, 
                                      on = 'user_name', how = 'left')

# Top users by tweets amount
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
sns.set_style("whitegrid")
fig.suptitle("Top users by tweets amount", size = 15)

sns.barplot(x = trump_agg['count'][:10], y = trump_agg['user_name'][:10],
            color = 'red', edgecolor = 'black', alpha = 0.8, ax = ax1)
ax1.get_xaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('')
ax1.set_ylabel('User name')
ax1.set_title('Trump', size = 13)


sns.barplot(x = biden_agg['count'][:10], y = biden_agg['user_name'][:10],
            color = 'blue', edgecolor = 'black', alpha = 0.8, ax = ax3)
ax3.get_xaxis().get_major_formatter().set_scientific(False)
ax3.set_xlabel('')
ax3.set_ylabel('User name')
ax3.set_title('Biden', size = 13)


sns.barplot(x = trump_top_authors_by_count['user_followers_count'], 
            y = trump_top_authors_by_count['user_name'],
            color = 'red', edgecolor = 'black', alpha = 0.8, ax = ax2)
ax2.get_xaxis().get_major_formatter().set_scientific(False)
ax2.get_yaxis().set_visible(False)
ax2.set_xlabel('')
ax2.set_title('User followers count', size = 13)


sns.barplot(x = biden_top_authors_by_count['user_followers_count'], 
            y = biden_top_authors_by_count['user_name'],
            color = 'blue', edgecolor = 'black', alpha = 0.8, ax = ax4)
ax4.get_xaxis().get_major_formatter().set_scientific(False)
ax4.get_yaxis().set_visible(False)
ax4.set_xlabel('')
ax4.set_title('User followers count', size = 13)

fig.show()

Users who posted the most tweets have a small amount of followers.

In [ ]:
# User followers count
trump_top_authors_by_likes = pd.merge(trump_agg.sort_values('likes', ascending = False)[:10], 
                                      trump_user_followers, on = 'user_name', how = 'left')
biden_top_authors_by_likes = pd.merge(biden_agg.sort_values('likes', ascending = False)[:10], 
                                      biden_user_followers, on = 'user_name', how = 'left')

# Top users by likes
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
sns.set_style("whitegrid")
fig.suptitle('Top users by likes', size = 15)

sns.barplot(x = trump_top_authors_by_likes['likes'], 
            y = trump_top_authors_by_likes['user_name'],
            color = 'red', edgecolor = 'black', alpha = 0.8, ax = ax1)
ax1.get_xaxis().get_major_formatter().set_scientific(False)
ax1.set_xlabel('')
ax1.set_ylabel('User name')
ax1.set_title('Trump', size = 13)


sns.barplot(x = biden_top_authors_by_likes['likes'], 
            y = biden_top_authors_by_likes['user_name'],
            color = 'blue', edgecolor = 'black', alpha = 0.8, ax = ax3)
ax3.get_xaxis().get_major_formatter().set_scientific(False)
ax3.set_xlabel('')
ax3.set_ylabel('User name')
ax3.set_title('Biden', size = 13)


sns.barplot(x = trump_top_authors_by_likes['user_followers_count'], 
            y = trump_top_authors_by_likes['user_name'],
            color = 'red', edgecolor = 'black', alpha = 0.8, ax = ax2)
ax2.xaxis.set_major_formatter(FuncFormatter(millions))
ax2.get_yaxis().set_visible(False)
ax2.set_xlabel('')
ax2.set_title('User followers count', size = 13)


sns.barplot(x = biden_top_authors_by_likes['user_followers_count'], 
            y = biden_top_authors_by_likes['user_name'],
            color = 'blue', edgecolor = 'black', alpha = 0.8, ax = ax4)
ax4.xaxis.set_major_formatter(FuncFormatter(millions))
ax4.get_yaxis().set_visible(False)
ax4.set_xlabel('')
ax4.set_title('User followers count', size = 13)

fig.show()

On the contrary, users whose tweets got many likes have far more followers, that looks logical. We will be able to classify these users into upholders or haters after sentiment analysis.

At first, we have to clean the data up. We'll use a great package [re](https://docs.python.org/3/library/re.html) for this. Let's look at the random tweets.

In [ ]:
trump['tweet'][19529]

In [ ]:
biden['tweet'][11650]

As intended, tweets have various kinds of noise, like emojis, special symbols, newline tabulators, links, etc. Also, not all tweets wrote in English (like the tweet above).

In [ ]:
# Tweet cleaner
def tweet_cleaner(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
trump_cleaned = trump.copy()
biden_cleaned = biden.copy()

trump_cleaned['tweet'] = trump_cleaned['tweet'].apply(lambda x: tweet_cleaner(x))
biden_cleaned['tweet'] = biden_cleaned['tweet'].apply(lambda x: tweet_cleaner(x))

In [ ]:
trump_cleaned['tweet'][19529]

In [ ]:
biden_cleaned['tweet'][11650]

Take a look at the length of tweets.

In [ ]:
trump_tweet_length = trump_cleaned.tweet.str.len()
biden_tweet_length = biden_cleaned.tweet.str.len()

sns.set_style("whitegrid")
plt.figure(figsize=(14, 5))

sns.distplot(trump_tweet_length, label = 'Trump', color = 'red', kde = False)
sns.distplot(biden_tweet_length, label = 'Biden', color = 'blue', kde = False)
plt.legend(prop={'size': 14})
plt.title('Tweet length', size = 15)
plt.xlabel('Length of tweet (symbols)')
plt.show()

We see that on average, the distributions of tweet length for both candidates are the same.

In [ ]:
trump_words = trump_cleaned.tweet.str.split().map(lambda x: len(x))
biden_words = biden_cleaned.tweet.str.split().map(lambda x: len(x))

sns.set_style("whitegrid")
plt.figure(figsize=(14, 5))

sns.distplot(trump_words, label = 'Trump', color = 'red', kde = False)
sns.distplot(biden_words, label = 'Biden', color = 'blue', kde = False)
plt.legend(prop={'size': 14})
plt.title('The number of words in tweet', size = 15)
plt.xlabel('Words')
plt.show()

Almost the same. 

In [ ]:
trump_word_len = trump_cleaned.tweet.str.split().apply(lambda x: [len(i) for i in x]).map(lambda x: np.mean(x))
biden_word_len = biden_cleaned.tweet.str.split().apply(lambda x: [len(i) for i in x]).map(lambda x: np.mean(x))

sns.set_style("whitegrid")
plt.figure(figsize=(14, 5))

sns.distplot(trump_word_len, label = 'Trump', color = 'red', kde = False)
sns.distplot(biden_word_len, label = 'Biden', color = 'blue', kde = False)
plt.legend(prop={'size': 14})
plt.title('The average word length', size = 15)
plt.xlabel('Length of word')
plt.show()

The chart above shows that almost all words have a range of lengths from 0 to 20. But there are some words with zero-length (NaN values), and, on the contrary, far longer than 20 letters (80, 100, and even more!). We'll remove only NaN-type outliers because long words are unique and don't have a powerful effect on data.

In [ ]:
# Tweets with NaN length
trump_NaN_len = trump_word_len[trump_word_len.isnull()].index
biden_NaN_len = biden_word_len[biden_word_len.isnull()].index

# removing tweets with NaN length
trump_cleaned = trump_cleaned.drop(trump_NaN_len, axis = 0)
biden_cleaned = biden_cleaned.drop(biden_NaN_len, axis = 0)

In [ ]:
# creating word lists
trump_corpus = []
words = trump_cleaned['tweet'].str.split().values.tolist()
trump_corpus = [word for i in words for word in i]

biden_corpus = []
words = biden_cleaned['tweet'].str.split().values.tolist()
biden_corpus = [word for i in words for word in i]

In [ ]:
# word frequency
trump_counter = Counter(trump_corpus)
trump_most = trump_counter.most_common()

biden_counter = Counter(biden_corpus)
biden_most = biden_counter.most_common()

In [ ]:
# The list of languages
stopwords.fileids()

In [ ]:
# Stopwords; we use English and Spanish as the most frequent in our data 
stop = set(np.concatenate((stopwords.words('english'), stopwords.words('spanish'))))

trump_x, trump_y = [], []
for word, count in trump_most[:100]:
    if word not in stop:
        trump_x.append(word)
        trump_y.append(count)
        
biden_x, biden_y = [], []
for word, count in biden_most[:100]:
    if word not in stop:
        biden_x.append(word)
        biden_y.append(count)

In [ ]:
# TOP-20 words for both candidates
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
sns.set_style("whitegrid")
plt.suptitle('TOP-20 words', size = 15)

sns.barplot(x = trump_y[:20], y = trump_x[:20], edgecolor = 'black', color = 'red', ax = ax1)
ax1.set_title('Trump', size = 13)
ax1.get_xaxis().get_major_formatter().set_scientific(False)

sns.barplot(x = biden_y[:20], y = biden_x[:20], edgecolor = 'black', color = 'blue', ax = ax2)
ax2.set_title('Biden', size = 13)
ax2.get_xaxis().get_major_formatter().set_scientific(False)

fig.show()

Among the most frequent words in tweets dedicated to Donald Trump (excluding candidates' proper nouns) occur both popular election words: "vote", "election", "president", "people", "Election Day", etc., and specific, like "MAGA" (Trump's tagline "Make America Great Again") or "die" (a word with negative sense). Specific words of tweets dedicated to Joe Biden: "Kamala Harris" (Vice President-elect of the United States), "BidenHarris", "win" (a word that is more frequent regarding Joe Biden than Donald Trump). Let's look at Bi and Tri n-grams of words.

In [ ]:
# ngrams of words
def get_top_ngram(corpus, n = None):
    vec = CountVectorizer(stop_words = stop, ngram_range = (n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq

### Top-10 bigrams

In [ ]:
# Data for bigrams
top_trump_n_bigrams = get_top_ngram(trump_cleaned['tweet'], 2)[:10]
trump_x, trump_y = map(list, zip(*top_trump_n_bigrams))

top_biden_n_bigrams = get_top_ngram(biden_cleaned['tweet'], 2)[:10]
biden_x, biden_y = map(list, zip(*top_biden_n_bigrams))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
sns.set_style("whitegrid")
plt.suptitle('TOP Bigrams', size = 15)

sns.barplot(x = trump_y, y = trump_x, edgecolor = 'black', color = 'red', ax = ax1)
ax1.set_title('Trump', size = 13)
ax1.get_xaxis().get_major_formatter().set_scientific(False)

sns.barplot(x = biden_y, y = biden_x, edgecolor = 'black', color = 'blue', ax = ax2)
ax2.set_title('Biden', size = 13)
ax2.get_xaxis().get_major_formatter().set_scientific(False)

fig.show()

### Top-10 trigrams

In [ ]:
# Data for trigrams
top_trump_n_bigrams = get_top_ngram(trump_cleaned['tweet'], 3)[:10]
trump_x, trump_y = map(list, zip(*top_trump_n_bigrams))

top_biden_n_bigrams = get_top_ngram(biden_cleaned['tweet'], 3)[:10]
biden_x, biden_y = map(list, zip(*top_biden_n_bigrams))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
sns.set_style("whitegrid")
plt.suptitle('TOP Trigrams', size = 15)

sns.barplot(x = trump_y, y = trump_x, edgecolor = 'black', color = 'red', ax = ax1)
ax1.set_title('Trump', size = 13)
ax1.get_xaxis().get_major_formatter().set_scientific(False)

sns.barplot(x = biden_y, y = biden_x, edgecolor = 'black', color = 'blue', ax = ax2)
ax2.set_title('Biden', size = 13)
ax2.get_xaxis().get_major_formatter().set_scientific(False)

fig.show()

### Wordclouds

In [ ]:
def show_wordcloud(data, title = None, color = 'white'):
    wordcloud = WordCloud(background_color=color,
                         stopwords=stop,
                         max_words=10000,
                         scale=3,
                         width = 4000, 
                         height = 2000,
                         collocations=False,
                         random_state=1)
    
    wordcloud = wordcloud.generate(str(data))
    
    plt.figure(1, figsize=(16, 8))
    plt.title(title, size = 15)
    plt.axis('off')
    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(trump_cleaned['tweet'].dropna(), title = 'Trump wordcloud', color = 'black')

In [ ]:
show_wordcloud(biden_cleaned['tweet'].dropna(), title = 'Biden wordcloud', color = 'black')

Now, it's time for **sentiment analysis**.

# **Sentiment analysis**
We'll use two tools: TextBlob package, and VADER from NLTK package.

In [ ]:
# Text polarity function
def polarity(data):
    return TextBlob(data).sentiment.polarity

trump_cleaned['polarity'] = trump_cleaned['tweet'].apply(lambda x: polarity(x))
biden_cleaned['polarity'] = biden_cleaned['tweet'].apply(lambda x: polarity(x))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
sns.set_style("whitegrid")
plt.suptitle('Polarity (TextBlob)', size = 15)
plt.text(x=-2.3, y=20.3, s='(from -1 (extremely negative) to 1 (extremely positive))', fontsize=10, alpha=0.8)


sns.kdeplot(trump_cleaned['polarity'], color = 'red', ax = ax1, legend = False)
ax1.set_title('Trump', size = 13)

sns.kdeplot(biden_cleaned['polarity'], color = 'blue', ax = ax2, legend = False)
ax2.set_title('Biden', size = 13)

fig.show()

Let's take a look at some extremely positive and extremely negative tweets of both candidates.

In [ ]:
# Trump negative tweets
trump_cleaned[trump_cleaned['polarity'] == -1]['tweet'][:5]

In [ ]:
# Trump positive tweets
trump_cleaned[trump_cleaned['polarity'] == 1]['tweet'][:5]

In [ ]:
# Biden negative tweets
biden_cleaned[biden_cleaned['polarity'] == -1]['tweet'][:5]

In [ ]:
# Biden positive tweets
biden_cleaned[biden_cleaned['polarity'] == 1]['tweet'][:5]

Not perfect (because the data still dirty, and the TextBlob method not 100% effective), but it works well enough. Let's try VADER method that works better with negative sentiment.

In [ ]:
# VADER Analyzer
sid = SentimentIntensityAnalyzer()

# Text polarity function
def get_vader_score(data):
    return sid.polarity_scores(data)['compound']

trump_cleaned['VADER'] = trump_cleaned['tweet'].apply(lambda x: get_vader_score(x))
biden_cleaned['VADER'] = biden_cleaned['tweet'].apply(lambda x: get_vader_score(x))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
sns.set_style("whitegrid")
plt.suptitle('Polarity (VADER)', size = 15)
plt.text(x=-2.4, y=9.3, s='(from -1 (extremely negative) to 1 (extremely positive))', fontsize=10, alpha=0.8)


sns.kdeplot(trump_cleaned['VADER'], color = 'red', ax = ax1, legend = False)
ax1.set_title('Trump', size = 13)

sns.kdeplot(biden_cleaned['VADER'], color = 'blue', ax = ax2, legend = False)
ax2.set_title('Biden', size = 13)

fig.show()

Yes, with VADER the percentage of neutral polarities (close to zero) decreased in the negative direction.

In [ ]:
print(trump_cleaned['tweet'].loc[945451])
print('*'*20)
print(trump_cleaned[['polarity', 'VADER']].loc[945451])

In [ ]:
print(biden_cleaned['tweet'].loc[296157])
print('*'*20)
print(biden_cleaned[['polarity', 'VADER']].loc[296157])

Wow, VADER really looks better at recognizing negative sentiment.

In [ ]:
# create 'date' feature
trump_cleaned['date'] = pd.to_datetime(trump_cleaned['created_at']).dt.date
biden_cleaned['date'] = pd.to_datetime(biden_cleaned['created_at']).dt.date

In [ ]:
def create_line_plot(data1, data2, FUN, ax, title):
    sns.lineplot(data = data1.groupby('date')[FUN].mean(), 
                 label = 'Trump', color = 'red', linewidth = 3, ax = ax)
    sns.lineplot(data = data2.groupby('date')[FUN].mean(), 
                 label = 'Biden', color = 'blue', linewidth = 3, ax = ax)
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %b'))
    ax.set_title(title, size = 13)
    ax.legend(prop={'size': 14})
    ax.set_ylabel('Polarity')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
sns.set_style("whitegrid")
plt.suptitle('Sentiment polarity changing', size = 15)

create_line_plot(trump_cleaned, biden_cleaned, 'polarity', ax1, '(TextBlob)')
create_line_plot(trump_cleaned, biden_cleaned, 'VADER', ax2, '(VADER)')

fig.show()

It looks like tweets dedicated to Joe Biden were more positive throughout the studied period. But these are average values. We should look at the changes in the polarity of positive and negative tweets separately.

In [ ]:
# Working datasets
# Trump
tb_pos_tr = trump_cleaned[trump_cleaned['polarity'] > 0]
tb_neg_tr = trump_cleaned[trump_cleaned['polarity'] < 0]
vader_pos_tr = trump_cleaned[trump_cleaned['VADER'] > 0]
vader_neg_tr = trump_cleaned[trump_cleaned['VADER'] < 0]

# Biden
tb_pos_bi = biden_cleaned[biden_cleaned['polarity'] > 0]
tb_neg_bi = biden_cleaned[biden_cleaned['polarity'] < 0]
vader_pos_bi = biden_cleaned[biden_cleaned['VADER'] > 0]
vader_neg_bi = biden_cleaned[biden_cleaned['VADER'] < 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
sns.set_style("whitegrid")
plt.suptitle('Positive polarity changing', size = 15)

create_line_plot(tb_pos_tr, tb_pos_bi, 'polarity', ax1, '(TextBlob)')
create_line_plot(vader_pos_tr, vader_pos_bi, 'VADER', ax2, '(VADER)')

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
sns.set_style("whitegrid")
plt.suptitle('Negative polarity changing', size = 15)

create_line_plot(tb_neg_tr, tb_neg_bi, 'polarity', ax1, '(TextBlob)')
create_line_plot(vader_neg_tr, vader_neg_bi, 'VADER', ax2, '(VADER)')

fig.show()

It's time for sentiment classification of users.

In [ ]:
trump_users_pol = trump_cleaned.groupby('user_name')['polarity', 'VADER'] \
    .mean().sort_values('polarity', ascending = False)

biden_users_pol = biden_cleaned.groupby('user_name')['polarity', 'VADER'] \
    .mean().sort_values('polarity', ascending = False)

In [ ]:
# Datasets
trump_top_authors_by_count = pd.merge(trump_top_authors_by_count, 
                                      trump_users_pol, on = 'user_name', 
                                      how = 'left').sort_values('VADER', ascending = True)

biden_top_authors_by_count = pd.merge(biden_top_authors_by_count, 
                                      biden_users_pol, on = 'user_name', 
                                      how = 'left').sort_values('VADER', ascending = True)

trump_top_authors_by_likes = pd.merge(trump_top_authors_by_likes, 
                                      trump_users_pol, on = 'user_name', 
                                      how = 'left').sort_values('VADER', ascending = True)

biden_top_authors_by_likes = pd.merge(biden_top_authors_by_likes, 
                                      biden_users_pol, on = 'user_name', 
                                      how = 'left').sort_values('VADER', ascending = True)

In [ ]:
def create_lollipop_plot(data, ax, first_color, second_color, title):
    ax.hlines(y = data['user_name'], xmin = data['polarity'], 
               xmax = data['VADER'], color = first_color, alpha = 0.5)
    ax.scatter(data['polarity'], data['user_name'], color = first_color, 
                alpha = 1, edgecolors = 'black', label = 'TextBlob')
    ax.scatter(data['VADER'], data['user_name'], color = second_color, 
                alpha = 1, edgecolors = 'black', label = 'VADER')
    ax.legend()
    ax.set_xlim(-1, 1)
    ax.axvline(x = 0, linestyle = '--', color = 'black', linewidth = 0.7)
    ax.set_title(title)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
sns.set_style("whitegrid")
fig.suptitle('Trump users sentiment polarity', size = 15)

create_lollipop_plot(trump_top_authors_by_count, ax1, 'red', 'orange',
                     'TOP users by tweets count')

create_lollipop_plot(trump_top_authors_by_likes, ax2, 'red', 'orange',
                     'TOP users by likes')

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
sns.set_style("whitegrid")
fig.suptitle('Biden users sentiment polarity', size = 15)

create_lollipop_plot(biden_top_authors_by_count, ax1, 'blue', 'green',
                     'TOP users by tweets count')

create_lollipop_plot(biden_top_authors_by_likes, ax2, 'blue', 'green',
                     'TOP users by likes')

fig.show()

And also we should look at the sentiment polarity of TOP-countries.

In [ ]:
# Datasets
tr_top_countries = trump_cleaned[trump_cleaned.country.isin(trump_tweets_countries.index)]
bi_top_countries = biden_cleaned[biden_cleaned.country.isin(biden_tweets_countries.index)]

trump_countries_pol = tr_top_countries.groupby('country')['polarity', 'VADER'] \
    .mean().sort_values('polarity', ascending = True)

biden_countries_pol = bi_top_countries.groupby('country')['polarity', 'VADER'] \
    .mean().sort_values('polarity', ascending = True)

In [ ]:
def create_lollipop_plot_2(data, ax, first_color, second_color, title):
    ax.hlines(y = data.index, xmin = data['polarity'], 
               xmax = data['VADER'], color = first_color, alpha = 0.5)
    ax.scatter(data['polarity'], data.index, color = first_color, 
                alpha = 1, edgecolors = 'black', label = 'TextBlob')
    ax.scatter(data['VADER'], data.index, color = second_color, 
                alpha = 1, edgecolors = 'black', label = 'VADER')
    ax.legend()
    ax.set_xlim(-1, 1)
    ax.axvline(x = 0, linestyle = '--', color = 'black', linewidth = 0.7)
    ax.set_title(title)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
sns.set_style("whitegrid")
fig.suptitle('Sentiment polarity by country', size = 15)

create_lollipop_plot_2(trump_countries_pol, ax1, 'red', 'orange',
                     'Trump')

create_lollipop_plot_2(biden_countries_pol, ax2, 'blue', 'green',
                     'Biden')

fig.show()

So, we've taken a quick look at the sentiment of tweets. There are a lot of analysis variants. It looks great to study the polarity of tweets by cities, or states, for instance, but these features have a lot of NaN values, and therefore don't cover the actual situation. You free to implement your own ideas (maybe I'll expand my own analysis late, this notebook is still in work). I hope you found this notebook useful. See you!